# Convert EBM to IRM

This notebook takes the three-layer energy balance model tunings from Donald Cummins and converts them to a three-layer impulse response function.

It will then save these into a CSV file.

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as pl
import pandas as pd
import scipy.linalg
import json
from tqdm import tqdm

In [ ]:
df = pd.read_csv(
    os.path.join("..", "data", "calibration", "4xCO2_cummins.csv")
)

In [ ]:
models = df['model'].unique()
models

In [ ]:
params = {}
for model in models:
    params[model] = {}
    for run in df.loc[df['model']==model, 'run']:
        condition = (df['model']==model) & (df['run']==run)
        params[model][run] = {}
        params[model][run]['gamma_autocorrelation'] = df.loc[condition, 'gamma'].values[0]
        params[model][run]['ocean_heat_capacity'] = df.loc[condition, 'C1':'C3'].values.squeeze()
        params[model][run]['ocean_heat_transfer'] = df.loc[condition, 'kappa1':'kappa3'].values.squeeze()
        params[model][run]['deep_ocean_efficacy'] = df.loc[condition, 'epsilon'].values[0]
        params[model][run]['sigma_eta'] = df.loc[condition, 'sigma_eta'].values[0]
        params[model][run]['sigma_xi'] = df.loc[condition, 'sigma_xi'].values[0]
        params[model][run]['forcing_4co2'] = df.loc[condition, 'F_4xCO2'].values[0]

In [ ]:
params

In [ ]:
# TODO: move to a constants module

EARTH_RADIUS = 6.371e6  # m
SECONDS_PER_YEAR = 365.2425 * 24 * 60 * 60  # we can be cleverer about this
DOUBLING_TIME_1PCT = np.log(2)/np.log(1.01)  # about 69.7

In [ ]:
def emergent_parameters(response_coeffs, timescales, forcing_4co2, forcing_2co2_4co2_ratio=0.476304):
    ecs = forcing_4co2 * forcing_2co2_4co2_ratio * np.sum(response_coeffs)
    tcr = forcing_4co2 * forcing_2co2_4co2_ratio * np.sum(
        response_coeffs*(
            1 - timescales/DOUBLING_TIME_1PCT * (
                1 - np.exp(-DOUBLING_TIME_1PCT/timescales)
            )
        )
    )
    return ecs, tcr

In [ ]:
def ebm3_to_irm3(params):
    """Converts the three-layer energy balance to impulse response form.
    
    Inputs
    ------
    params : dict
    
    Returns
    -------
    params : dict
    """
    
    # unpack parameters
    ocean_heat_capacity = params['ocean_heat_capacity']
    ocean_heat_transfer = params['ocean_heat_transfer']
    deep_ocean_efficacy = params['deep_ocean_efficacy']
    forcing_4co2 = params.get('forcing_4co2', None)
    
    nbox = len(ocean_heat_capacity)  # should be 3
    
    # Define the matrix of differential equations
    eb_matrix = np.array(
        [
            [
                -(ocean_heat_transfer[0]+ocean_heat_transfer[1])/ocean_heat_capacity[0],
                ocean_heat_transfer[1]/ocean_heat_capacity[0], 
                0
            ],
            [
                ocean_heat_transfer[1]/ocean_heat_capacity[1],
                -(ocean_heat_transfer[1]+deep_ocean_efficacy*ocean_heat_transfer[2])/ocean_heat_capacity[1],
                deep_ocean_efficacy*ocean_heat_transfer[2]/ocean_heat_capacity[1]
            ],
            [
                0, 
                ocean_heat_transfer[2]/ocean_heat_capacity[2],
                -ocean_heat_transfer[2]/ocean_heat_capacity[2]
            ]
        ]
    )
    
    # calculate the eigenvectors and eigenvalues, these are the timescales of responses
    eb_matrix_eigenvalues, eb_matrix_eigenvectors = scipy.linalg.eig(eb_matrix)
    timescales = -1/(np.real(eb_matrix_eigenvalues))
    response_coefficients = timescales * (eb_matrix_eigenvectors[0,:] * scipy.linalg.inv(eb_matrix_eigenvectors)[:,0]) / ocean_heat_capacity[0]

    # calculate ECS and TCR from this parameter set; if 4xCO2 forcing is given
    if forcing_4co2 is not None:
        params['ecs'], params['tcr'] = emergent_parameters(response_coefficients, timescales, forcing_4co2)
    
    params['timescales'] = timescales
    params['response_coefficients'] = response_coefficients
    return params

In [ ]:
for model in models:
    for run in df.loc[df['model']==model, 'run']:
        condition = (df['model']==model) & (df['run']==run)
        ebm3_to_irm3(params[model][run])

In [ ]:
params

In [ ]:
# reconstruct a data table and save
df_out = pd.DataFrame(columns=['model', 'run', 'ecs', 'tcr', 'tau1', 'tau2', 'tau3', 'q1', 'q2', 'q3'])

#values_to_add = {'A': 1, 'B': 2}
#row_to_add = pd.Series(values_to_add, name='x')

#df = df.append(row_to_add)

count = 0
for model in models:
    for run in df.loc[df['model']==model, 'run']:
        values_to_add = {
            'model': model,
            'run': run,
            'ecs': params[model][run]['ecs'],
            'tcr': params[model][run]['tcr'],
            'tau1': params[model][run]['timescales'][0],
            'tau2': params[model][run]['timescales'][1],
            'tau3': params[model][run]['timescales'][2],
            'q1': params[model][run]['response_coefficients'][0],
            'q2': params[model][run]['response_coefficients'][1],
            'q3': params[model][run]['response_coefficients'][2],
        }
        row_to_add = pd.Series(values_to_add, name=count)
        df_out = df_out.append(row_to_add)
        count = count + 1

In [ ]:
df_out

In [ ]:
df_out.to_csv(os.path.join("..", "data", "calibration", "4xCO2_impulse_response.csv"))